# Build a Large language Model from scratch y sebastian Raschka
## Chapter 1
- LLMs have upended natural language processing
- use new deep learning approaches over explicit rules and statistical methods
- better at understanding, generating and translating human language
- modern llms are first pretrained on a large body of unabeled text to predict the next word in a sentence as a label
- then fine-tuned on a smaller, labeled target dataset to follow instructions or perform classification tasks
- llms are based on the transformer architecture
- the attentoin mechanism gives the llm ability to "focus" on specific parts of the input while making one output at a time
- datasets of billions of words are needed for llm pretraining
- while only trainedto predect next words
- emergent properties to classify translate or summarize texts happen too
- once pretrained the result is a foundation model
- this can be finetuned to be more efficient for different downstream tasks
- llms finetuned for specific tasks can outperform general purpose llms on specific tasks